# Generating counterfactual explanations with any ML model

The goal of this notebook is to show how to generate CFs for ML models using frameworks other than TensorFlow or PyTorch. This is a work in progress and here we show a method to generate diverse CFs by independent random sampling of features. We use scikit-learn models for demonstration.  

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in dice_ml.utils.helpers module.

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

## Training a custom ML model

Below, we build an Artificial Neural Network using *MLPClassifier* in scikit-learn. We try to use the same set of parameters as used in this advanced [notebook](DiCE_with_private_data.ipynb), however, there are other framework-dependent parameters that can't be easily ported, so the accuracy/performance of the two models will be different.

In [5]:
train, test = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']
X_test = test.loc[:, test.columns != 'income']
y_test = test.loc[:, test.columns == 'income']

In [6]:
mlp = MLPClassifier(hidden_layer_sizes=(20), alpha=0.001, learning_rate_init=0.01, batch_size=32, random_state=17,
                    max_iter=20, verbose=False, validation_fraction=0.2, ) #max_iter is epochs in TF
mlp.fit(X_train, y_train.values.ravel())

c:\users\t-rakom\python-virtual-environments\dice_model_change\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(alpha=0.001, batch_size=32, hidden_layer_sizes=20,
              learning_rate_init=0.01, max_iter=20, random_state=17,
              validation_fraction=0.2)

In [7]:
# provide the trained ML model to DiCE's model object
backend = None
m = dice_ml.Model(model=mlp, backend=backend)

## Generate diverse counterfactuals

In [8]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [11]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,47.0,Self-Employed,Some-college,Married,Professional,-,-,19.0,0.326
1,66.0,Government,Prof-school,Separated,Other/Unknown,Other,Male,81.0,0.29
2,33.0,Government,Doctorate,Separated,Sales,Other,Male,50.0,0.479
3,66.0,-,Assoc,Married,Professional,-,-,64.0,0.453


It can be observed that the random sampling method produces less sparse CFs in contrast to current DiCE's implementation. The sparsity issue with random sampling worsens with increasing *total_CFs* 

Further, different sets of counterfactuals can be generated with different random seeds.

In [13]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", random_seed=9) # default ranomd see is 17

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [14]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,76.0,Self-Employed,Some-college,Married,Professional,-,-,-,0.356
1,69.0,Government,Prof-school,Separated,Sales,Other,Male,57.0,0.467
2,63.0,Government,Doctorate,Separated,Sales,Other,Male,80.0,0.343
3,33.0,Self-Employed,Some-college,Married,Professional,-,-,68.0,0.306


### Selecting the features to vary

When few features are fixed, random sampling is unable to generate valid CFs while we get valid diverse CFs with current DiCE.

In [15]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       features_to_vary=['workclass','education','occupation','hours_per_week'])

Only 0 (required 4) Diverse Counterfactuals found for the given configuation, perhaps try with different values of proximity (or diversity) weights or learning rate... ; total time taken: 00 min 00 sec


In [16]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Assoc,-,White-Collar,-,-,83.0,0.969
1,-,Self-Employed,Some-college,-,-,-,-,22.0,0.998
2,-,Other/Unknown,-,-,Blue-Collar,-,-,50.0,0.997
3,-,Other/Unknown,-,-,Blue-Collar,-,-,69.0,0.994


### Choosing feature ranges

Since the features are sampled randomly, they can freely vary across their range. In the below example, we show how range of continuous features can be controlled using *permitted_range* parameter that can now be passed during CF generation.

In [17]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       permitted_range={'age':[22,50],'hours_per_week':[40,60]})

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [18]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.997482



Diverse Counterfactual set (new outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,27.0,Government,Doctorate,Separated,Sales,Other,Male,56.0,0.493
1,40.0,Government,Doctorate,Separated,Sales,Other,Male,58.0,0.421
2,32.0,Self-Employed,Some-college,Married,Professional,-,-,47.0,0.309
3,39.0,-,Assoc,Married,Professional,-,-,43.0,0.376
